***리뷰 뽑아오는 코드***

***requests,json으로 한 최종코드***

In [181]:
import time
import pandas as pd
import numpy as np
import requests
import json
import os

# json파일 불러오기 ==> json파일 다운로드 및 경로는 바꿔야될 수 있습니다. 
file_path='./goodsNo_dict.json'
with open(file_path, 'r', encoding='utf-8-sig') as file:
    goodsNo_json = json.load(file)

# review와 관련된 정보들을 가져올 dictionary
review_dict = {
    'brand_name' : [], # 브랜드 이름
    'goodsNo' : [], # 상품 고유 번호
    'gdasCont' : [], # 상품 리뷰
    'goodsNm' : [], # 상품 이름
    'dispRegDate' : [], # 리뷰날짜
    'gdasSeq' : [], # 리뷰 구분하기 위한 리뷰 순서?
    'subcategory' : [], # 상품 카테고리 번호 
#     'review_star' : []  # 리뷰 별점 
}

start_time = time.time()
# brand_list = [‘닥터지‘, ‘라운드랩‘, ‘라네즈‘, ‘아이소이‘, ‘아누아‘, ‘에스트라’] # 올리브영 기준 top 5 스킨케어 브랜드 (0426 기준)
brand_list = ['닥터지', '라운드랩']  
for brand in brand_list:
    for sub_cat, goodsNo in goodsNo_json[brand].items(): 
        for goods_num in goodsNo: 
            page_num = 1
            while True:
                headers = {'X-Requested-With' : 'XMLHttpRequest'}
                tmp_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?'+\
                'goodsNo={}&gdasSort=02&itemNo=all_search&'.format(goods_num) +\
                'pageIdx={}&cTypeLength=0'.format(str(page_num)) 

                res = requests.get(tmp_url, headers=headers)
                crawled_data = json.loads(res.text)
                
                if crawled_data == {}:
                    break
                
                for review_idx in range(len(crawled_data['gdasList'])):
                    # review_dict에 결과물 넣기 
                    review_dict['goodsNo'].append(goods_num) # goodsNo
                    review_dict['goodsNm'].append(crawled_data['gdasList'][review_idx]['goodsNm']) # 상품이름
                    review_dict['gdasCont'].append(crawled_data['gdasList'][review_idx]['gdasCont']) # 상품리뷰
                    review_dict['dispRegDate'].append(crawled_data['gdasList'][review_idx]['dispRegDate']) # 상품리뷰 날짜
                    review_dict['gdasSeq'].append(crawled_data['gdasList'][review_idx]['gdasSeq']) # 상품 구별을 위한 seq number
                    review_dict['subcategory'].append(sub_cat) # subcategory 넘버
                    review_dict['brand_name'].append(brand) # 브랜드 이름 
                
                page_num+=1
                
                ##### csv파일로 만드는 코드 ####
                
                
                time.sleep(0.07)
                
end_time = time.time()
time_lapsed = end_time - start_time

df = pd.DataFrame(review_dict)
print(df.shape)

(50324, 7)


In [185]:
df.tail()

,brand_name,goodsNo,gdasCont,goodsNm,dispRegDate,gdasSeq,subcategory
50319,라운드랩,A000000150447,자주 사용하는 제품이예요\r\n가격도 저렴하고\r\n냄새도 좋아요,라운드랩 1025 독도 미스트 150ml,2021.05.21,8933855,100000100010010
50320,라운드랩,A000000150447,독도토너를 잘 사용하구 있어서 구입했는데 이제품 역시 순해서 좋네요. 그냥 딱 수분...,라운드랩 1025 독도 미스트 150ml,2021.05.20,8922590,100000100010010
50321,라운드랩,A000000150447,건조한 실내에서 미스트를 자주 사용하는데 촉촉하고 좋습니다.,라운드랩 1025 독도 미스트 150ml,2021.05.19,8915350,100000100010010
50322,라운드랩,A000000150447,친구 생일 선물로 구매했어요\r\n안개분사가 아니어서 조금 아쉽지만 성분이 괜찮으니...,라운드랩 1025 독도 미스트 150ml,2021.05.09,8793980,100000100010010
50323,라운드랩,A000000150447,얼굴에 닿는 화장품은 무조건 화해에서 성분 보고 구매를 하는 타입입니다. 미스트로 ...,라운드랩 1025 독도 미스트 150ml,2021.05.08,8779355,100000100010010


In [186]:
goodsNo_json

{'닥터지': {'100000100010008': ['A000000182369',
   'A000000174023',
   'A000000171651',
   'A000000163655',
   'A000000165624',
   'A000000179655',
   'A000000131849',
   'A000000182277',
   'A000000160395',
   'A000000154177',
   'A000000145578',
   'A000000170518',
   'A000000170038',
   'A000000165623',
   'A000000163656',
   'A000000149399',
   'A000000135930',
   'A000000140753',
   'A000000165710',
   'A000000166642',
   'A000000123503'],
  '100000100010009': ['A000000164615',
   'A000000174203',
   'A000000182586',
   'A000000151512',
   'A000000165621',
   'A000000171844',
   'A000000180436',
   'A000000172453',
   'A000000123505',
   'A000000170037',
   'A000000163653',
   'A000000163651',
   'A000000163440',
   'A000000152618',
   'A000000165711',
   'A000000174312',
   'A000000166472',
   'A000000147130',
   'A000000142567',
   'A000000180435',
   'A000000175772',
   'A000000154173',
   'A000000165694',
   'A000000147299',
   'A000000151991',
   'A000000129094',
   'A000000157